In [30]:
dataset_train = torch.load('/kaggle/input/orientation-dataset/train_dataset_all.pt')
dataset_valid = torch.load('/kaggle/input/orientation-dataset/train_dataset_all.pt')

<ipython-input-30-5714f3b20d5f>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset_train = torch.load('/kaggle/input/orientation-dataset/train_dataset_all.pt')
<ipytho

In [31]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch
from typing import List, Union
from transformers import AutoTokenizer, AutoModel, PreTrainedModel
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, BertForSequenceClassification
import pandas as pd

from sklearn.metrics import accuracy_score, confusion_matrix

def evaluate(dataset: Dataset, model: PreTrainedModel, device: torch.device = torch.device('cpu'), plot: bool = False):
    '''
    Evaluates the model on the given dataset.
    
    Parameters:
        dataset: Dataset
            The dataset to evaluate on.
        model: PreTrainedModel
            The model to evaluate.
        device: torch.device
            The device to use.
        plot: bool
    '''
    
    model.eval()
    loader = DataLoader(dataset, batch_size=16, shuffle=False)
    correct_labels = []
    model_predictions = []
    probs = []
    attentions = []
    embeddings = []
    texts = []
    with torch.no_grad():
        for batch in loader:
            id_, speaker, sex, text, text_en, embedding, attention_mask, label = batch
            texts.extend(text_en)
            embedding = embedding.to(device)
            attention_mask = attention_mask.to(device).squeeze(1)
            assert(attention_mask.nonzero().size() == embedding.nonzero().size())
            label = label.to(device)
            model_output = model(input_ids=embedding, labels=label, attention_mask=attention_mask, output_attentions=True)
            embeddings.extend(embedding.cpu())
            
            
            attention = torch.mean(model_output.attentions[-1], dim=1).squeeze()[:,0]
           
           
            attentions.extend(attention.cpu().numpy())
            logits = model_output.logits
            
            prob = torch.max(torch.softmax(logits, dim=1), dim=1)
            
            probs.extend(prob.values.cpu())
            predictions = torch.argmax(logits, dim=1)
            correct_labels.extend(label.cpu().numpy())
            model_predictions.extend(predictions.cpu().numpy())

    accuracy = accuracy_score(correct_labels, model_predictions)
    print(f'Accuracy: {accuracy}')
    print(f'Confusion matrix:\n{confusion_matrix(correct_labels, model_predictions)}')
    
    return correct_labels, model_predictions, probs, attentions, embeddings, texts




In [32]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = torch.load('/kaggle/input/distilbertforattention/pytorch/default/1/distilbert_cased_en_novi.pt')

<ipython-input-32-5f85fd89954e>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/kaggle/input/distilbertforattention/pytorch/default/1/distilbert_cased_

In [33]:
output = evaluate(dataset_valid,model, 'cuda:0')

Accuracy: 0.8838941771990704
Confusion matrix:
[[10812  1945]
 [ 1702 16952]]


In [35]:
labels, predictions, probs, attentions, embeddings, old_texts = output

In [36]:
labels = np.array(labels)
predictions = np.array(predictions)
probs = np.array(probs)
attentions = np.array(attentions)
embeddings = np.array(embeddings)
print(attentions.shape)
print(attentions.reshape(len(labels),-1).shape)

(31411, 512)
(31411, 512)


In [37]:
ind = labels == predictions
print(len(ind))
labels = labels[ind]
predictions = predictions[ind]
probs = probs[ind]
attentions = attentions[ind]
embeddings = embeddings[ind]
texts = []
for i, cond in enumerate(ind):
    if cond:
        texts.append(old_texts[i])
        

31411


In [38]:
most_left = torch.topk(torch.tensor(probs), 5000, largest=False).indices.numpy()
good = most_left[6]
most_important = torch.topk( torch.tensor(attentions[good]), 15).indices.numpy()

print(tokenizer.decode(embeddings[good][most_important]))
for ind in most_left[6:10]:
    print(texts[ind])
most_important = torch.topk( torch.tensor(attentions[most_left]), 5).indices.numpy()
print(most_important.shape)
print(most_left.shape)

counter_l = {}
for ind, att in zip(most_left, most_important):
    words = tokenizer.decode(embeddings[ind][att]).lower().split(' ')
    for word in words:
        if '[SEP]' in word:
            word = word.replace('[SEP]', '')
        if '....' in word:
            word = word.replace('....', '')
        if '...' in word:
            word = word.replace('...', '')
        if '..' in word:
            word = word.replace('..', '')
        if '.' in word:
            word = word.replace('.', '')
        if ',' in word:
            word = word.replace(',', '')
        if '–' in word:
            word = word.replace('–', '')
        if '?' in word:
            word = word.replace('?', '')
        if '!' in word:
            word = word.replace('!', '')
        if '""' in word:
            word = word.replace('""', '')
        if '</s>' in word:
            word = word.replace('</s>', '')
        if '<s>' in word:
            word = word.replace('<s>', '')
        if '[sep]' in word:
            word = word.replace('[sep]', '')
        if '[cls]' in word:
            word = word.replace('[cls]', '')
        if '>' in word:
            word = word.replace('>', '')
        if word not in counter_l:
            counter_l[word] = 1
        else:
            counter_l[word] += 1
            
print()
most_popular_left = list(map(lambda x: x,sorted(counter_l.items(), key=lambda x: -x[1])))

print(most_popular_left[:100])

[SEP] President.lsson. borders borders Sweden infection [CLS].am Swedish border.
Madam President, I would like Thank you, Annika Qarlsson, for a very urgent question! <p> We should not repeat the mistake that took place last spring. We in Sweden, as I mentioned earlier, have consistently been in favour of openness in relation to borders. We are also leaning on the evidence-based evidence available from the European Disease Control Agency and from our own public health authority, which says that border closures are not an effective means of combating the spread of infection. <p> The Commission has also said this. The Commission says that if borders are to be closed now, a common approach must be found to restrictions and to the level to be taken. <p> On the Swedish side, we are pushing for there to be no border closures at all. We don't have a lockdown yet. However, we have new shutdowns which are worrying, although we do not have production units that are closing down. What we are doin

In [39]:
most_right = torch.topk(torch.tensor(probs), 5000, largest=True).indices.numpy()

most_important = torch.topk( torch.tensor(attentions[most_right]), 5).indices.numpy()

counter_r = {}
for ind, att in zip(most_right, most_important):
    words = tokenizer.decode(embeddings[ind][att]).lower().split(' ')
    
    for word in words:
        if '[SEP]' in word:
            word = word.replace('[SEP]', '')
        if '....' in word:
            word = word.replace('....', '')
        if '...' in word:
            word = word.replace('...', '')
        if '..' in word:
            word = word.replace('..', '')
        if '.' in word:
            word = word.replace('.', '')
        if ',' in word:
            word = word.replace(',', '')
        if '–' in word:
            word = word.replace('–', '')
        if '?' in word:
            word = word.replace('?', '')
        if '!' in word:
            word = word.replace('!', '')
        if '""' in word:
            word = word.replace('""', '')
        if '</s>' in word:
            word = word.replace('</s>', '')
        if '<s>' in word:
            word = word.replace('<s>', '')
        if '[sep]' in word:
            word = word.replace('[sep]', '')
        if '[cls]' in word:
            word = word.replace('[cls]', '')
        if '>' in word:
            word = word.replace('>', '')
        if word not in counter_r:
            counter_r[word] = 1
        else:
            counter_r[word] += 1
            

most_popular_right = list(map(lambda x: x,sorted(counter_r.items(), key=lambda x: -x[1])))
print(most_popular_right[:100])


[('', 7074), ('will', 1443), ('minister', 1119), ('my', 1089), ('<', 922), ('government', 769), ('right', 736), ('are', 686), ('is', 611), ('the', 571), ('friend', 520), ('secretary', 515), ('noble', 316), ('our', 297), ('have', 243), ('does', 213), ('prime', 174), ('has', 148), ('thank', 142), ("'", 141), ('can', 134), ('chancellor', 124), ('salute', 113), ('that', 108), ('department', 106), ('now', 99), ('agree', 95), ('we', 92), ('must', 88), ('lords', 85), ('still', 82), ('you', 81), ('but', 66), ('he', 65), ('dear', 58), ('am', 57), ('need', 57), ('p', 55), ('want', 52), ('why', 52), ('president', 50), ('to', 50), ('ho', 49), ('all', 44), ('continue', 44), ('wish', 38), ('ministry', 38), ('home', 38), ('as', 33), ('very', 33), ('i', 32), ('ministers', 32), ('re', 32), ('ask', 31), ('country', 29), ('leader', 29), ('beloved', 28), ('assure', 28), ('this', 27), ('do', 26), ('what', 25), ('going', 24), ('yet', 23), ('much', 22), ('know', 21), ('mr', 21), ('and', 21), ('nation', 21), 

In [40]:
w = 'treasury'
print(counter_l[w])
print(counter_r[w])

1
7
